In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-0xUZ6aBpi14QW cA"



# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Analogical Prompting: The Power of Custom Examples to Improve LLMs

Analogical Prompting, introduced by Yasunaga et al. (2023), is a sophisticated method that extends the concept of In-Context Learning by automatically generating relevant examples enriched with reasoning.

## How Does Analogical Prompting Work?

Imagine a teacher who, faced with a student struggling with a complex mathematical problem, would spontaneously present an analogous but simpler example to illustrate the solution method.

Analogical Prompting reproduces this pedagogical approach in two key phases:

1. **Automatic example generation**: Instead of using predefined examples, the system dynamically generates exemplars analogous to the problem at hand, but with known or more accessible solutions.

2. **Inclusion of reasoning chains**: Each generated example is accompanied by a complete Chain-of-Thought (CoT) that explicitly details the step-by-step resolution process.

## Why Is This Particularly Effective?

This approach offers considerable advantages by allowing the model to:

* Benefit from examples specifically tailored to each problem
* Observe reasoning directly applicable to the case being studied
* Transfer resolution strategies from an analogous problem to the target problem
* Learn by example rather than through abstract instructions

The results are particularly impressive in:
* Complex mathematical reasoning
* Code generation
* Problem-solving requiring structured approaches

It's like having a tutor who, for each new challenge, would first show you how to solve a similar problem before letting you try.

This method is similar to the SG-ICL (Self-Generated In-Context Learning) approach, but distinguishes itself by focusing on generating specifically analogous examples and the systematic inclusion of detailed reasoning chains.

The next time you use AI to solve a complex problem, you might indirectly benefit from this ability to draw inspiration from automatically generated analogous examples!

In [4]:
analogical_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Analogical Prompting based on user questions.
    
    Analogical Prompting is a technique within Chain-of-Thought (CoT) that automatically generates exemplars with reasoning paths by drawing analogies between the current problem and similar problems. 
    
    This technique significantly improves performance on mathematical reasoning and code generation tasks by leveraging the power of analogical thinking and transferring problem-solving strategies from familiar domains to new ones.
    
    The Analogical Prompting process includes the following steps:
    1. Identify a similar problem: Find or create an analogous problem that shares structural similarities with the current problem.
    2. Solve the analogous problem: Work through the analogous problem step by step, showing the complete reasoning path.
    3. Transfer the solution strategy: Apply the same reasoning pattern from the analogous problem to the current problem.
    4. Answer the original question: Provide a final answer based on the transferred reasoning.
    
    Please structure your response in the following format:
    
    Step 1: [Analogous problem identification]
    Step 2: [Solution of the analogous problem]
    Step 3: [Transfer of solution strategy]
    Step 4: [Final Answer]
    
    Example:
    Question: "A warehouse has 25 rows of shelves with 12 boxes per shelf. If each box contains 8 items, how many total items are in the warehouse?"
    
    Step 1: Analogous problem identification:
    This problem involves multiplication across multiple levels of organization. Let me consider an analogous problem:
    "A school has 15 classrooms with 20 desks per classroom. If each desk has 2 drawers, how many total drawers are there in the school?"
    
    Step 2: Solution of the analogous problem:
    To solve this analogous problem:
    1. First, I need to find the total number of desks: 15 classrooms × 20 desks per classroom = 300 desks
    2. Then, I need to find the total number of drawers: 300 desks × 2 drawers per desk = 600 drawers
    So the school has a total of 600 drawers.
    
    Step 3: Transfer of solution strategy:
    Now I can apply the same strategy to our original problem about the warehouse:
    1. First, I need to find the total number of boxes: 25 rows × 12 boxes per shelf = 300 boxes
    2. Then, I need to find the total number of items: 300 boxes × 8 items per box = 2,400 items
    
    Step 4: The warehouse contains a total of 2,400 items.
    """),
    ("human", "{question}"),
    ("assistant", """
    Step 1: Analogous problem identification:
    [Identify or create a problem that has a similar structure or solution pattern to the current question. The analogous problem should be simpler or more familiar, but maintain the key structural elements of the original problem.]
    
    Step 2: Solution of the analogous problem:
    [Work through the analogous problem step by step, showing the complete reasoning path:
    1. [First step in solving the analogous problem]
    2. [Second step in solving the analogous problem]
    3. [Continue with additional steps as needed]
    ...]
    
    Step 3: Transfer of solution strategy:
    [Apply the reasoning pattern from the analogous problem to the original question:
    1. [First step in solving the original problem, based on the analogy]
    2. [Second step in solving the original problem, based on the analogy]
    3. [Continue with additional steps as needed]
    ...]
    
    Step 4: [Provide a clear, concise answer to the original question, drawing directly from the transferred solution strategy]
    """)
])

For Analogical Prompting, the most suitable questions are those that benefit from reasoning by analogy and strategy transfer. Here are types of questions that would be particularly well-suited to this approach:

### Mathematical Problems
- Algebra problems with similar structures (equations, systems of equations)
- Geometry problems requiring application of specific theorems
- Probability or statistics calculations with transferable methodologies
- Optimization or minimization/maximization problems

### Programming and Algorithms
- Coding challenges where similar patterns can be applied
- Algorithm implementation in different contexts
- Code optimization with comparable structures
- Debugging code with typical errors

### Logical Reasoning
- Logic puzzles with similar structures
- Sequence or logical series problems
- Deduction riddles where elimination methods are applicable
- Planning or scheduling problems

### Concrete Examples of Questions

1. "How do you write a recursive function to calculate the nth number in the Fibonacci sequence?"
   - An analogous problem could be recursive factorial calculation

2. "A tank fills in 8 hours and drains in 12 hours. If both taps are open, how long will it take to fill the tank?"
   - Analogous to relative speed problems or collaborative work problems

3. "How do you implement an efficient algorithm to find all anagrams in a list of words?"
   - Comparable to other string manipulation problems

4. "In a single-elimination tournament with 64 teams, how many matches will be played in total?"
   - Analogous to tree structure or combinatorial problems

5. "How do you solve the differential equation dy/dx = y² + x?"
   - Comparable to other differential equations with similar solution techniques

Analogical Prompting is particularly effective for these types of questions because it allows the LLM to build on its understanding of a similar but simpler or more familiar problem, then transfer that solution strategy to the original problem, facilitating learning and application of structured methods.

In [7]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the ThoT prompt and extract ONLY the final answer (Step 4)
def generate_analogical_prompt_node_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = analogical_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final answer from Step 4
    final_answer_pattern = r'Step 4:([\s\S]*?)(?=$|Step \d:)'
    final_answer_match = re.search(final_answer_pattern, full_response)
    
    if final_answer_match:
        final_solution = final_answer_match.group(1).strip()
    else:
        # Alternative pattern
        alternative_pattern = r'Step 4: \[Final Answer\]([\s\S]*?)(?=$|Step \d:)'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('analogical_prompt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL ANSWER (STEP 4) ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final answer in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response
    }

# Definition of state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_analogical_prompt", generate_analogical_prompt_node_node)

# Graph configuration
graph_builder.set_entry_point("generate_analogical_prompt")
graph_builder.add_edge("generate_analogical_prompt", END)
graph = graph_builder.compile()

# Usage example
inputs = {
    "messages": [HumanMessage(content="In a single-elimination tournament with 64 teams, how many matches will be played in total?")],
    "question": None,
    "full_response": None
}

# Print the question at the beginning
print("QUESTION:")
print(inputs["messages"][0].content)
print("\nFINAL ANSWER (STEP 4):")

# Graph execution - only display Step 4
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_analogical_prompt":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'analogical_prompt.txt')")

QUESTION:
In a single-elimination tournament with 64 teams, how many matches will be played in total?

FINAL ANSWER (STEP 4):
The total number of matches played in the tournament with 64 teams is 63.

(Complete results saved in 'analogical_prompt.txt')
